---
title: "Naïve Bayes"
author: Marion Bauman
format:
    html:
        embed-resources: true
---

## Introduction to Naïve Bayes

In this next section of the project, I will be using the Naïve Bayes algorithm to classify both textual and numerical data. The Naïve Bayes method is an algorithm based on Bayes' Theorem, which states that:

$$P(Y|X) = \frac{P(X|Y)P(Y)}{P(X)}$$.

From here we also know that 

$$P(Y|X) \propto P(X|Y)P(Y)$$.

Put differently, this mean that the probability of $Y$ given $X$ is proportional to the probability of $X$ given $Y$ times the probability of $Y$. 

Generally, machine learning algorithms aim to find $P(Y|X)$, that is, to predict $Y$ given all features $X$.